<a href="https://colab.research.google.com/github/kalehariprasad/customer_churn_prediction/blob/main/notebooks/custumer_churn_prediction_model_building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [ ]:
df=pd.read_csv("/content/Assignment_Data - ML_Intern_Assignment_Data.csv.csv")

In [ ]:
df.head()

,CustomerID,Gender,Age,Tenure,MonthlyCharges,TotalCharges,PaymentMethod,Churn,ServiceUsage1,ServiceUsage2,ServiceUsage3
0,CUST0001,Male,20,5,62.31,2950.41,Credit Card,Yes,59,20,63
1,CUST0002,Female,25,23,118.96,7460.55,Bank Transfer,No,42,41,136
2,CUST0003,Female,25,39,69.20,3064.73,Bank Transfer,No,65,0,165
3,CUST0004,Male,56,47,145.90,2612.22,Cash,No,64,40,169
4,CUST0005,Female,33,69,88.51,4833.99,Cash,No,62,1,54




*   based on our previous data oversight we will drop 	CustomerID coulmn



In [ ]:
df.drop(columns = 'CustomerID',inplace = True)
df.head()

,Gender,Age,Tenure,MonthlyCharges,TotalCharges,PaymentMethod,Churn,ServiceUsage1,ServiceUsage2,ServiceUsage3
0,Male,20,5,62.31,2950.41,Credit Card,Yes,59,20,63
1,Female,25,23,118.96,7460.55,Bank Transfer,No,42,41,136
2,Female,25,39,69.20,3064.73,Bank Transfer,No,65,0,165
3,Male,56,47,145.90,2612.22,Cash,No,64,40,169
4,Female,33,69,88.51,4833.99,Cash,No,62,1,54


In [ ]:
df.select_dtypes(exclude="object")

,Age,Tenure,MonthlyCharges,TotalCharges,ServiceUsage1,ServiceUsage2,ServiceUsage3
0,20,5,62.31,2950.41,59,20,63
1,25,23,118.96,7460.55,42,41,136
2,25,39,69.20,3064.73,65,0,165
3,56,47,145.90,2612.22,64,40,169
4,33,69,88.51,4833.99,62,1,54
...,...,...,...,...,...,...,...
195,32,61,113.54,9607.02,80,39,147
196,47,67,133.12,4380.96,21,46,62
197,61,9,96.09,1599.30,56,23,154
198,24,45,142.44,3527.95,69,15,118


In [ ]:
x=df.drop(columns = 'Churn')
y = df['Churn']
y= y.map({'Yes':1,'No':0})

In [ ]:
train_x, test_x, train_y, test_y=train_test_split(x,y, random_state=42, test_size=0.2)

In [ ]:
cat_cols = ['Gender','PaymentMethod']
numerical_cols_standard = ['Age', 'Tenure', 'MonthlyCharges', 'TotalCharges']
numerical_cols_minmax = ['ServiceUsage1', 'ServiceUsage2', 'ServiceUsage3']

In [ ]:
standard_scaler = StandardScaler()
min_max_scaler = MinMaxScaler()
one_hot_encoder = OneHotEncoder(sparse_output=True)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('standard', standard_scaler, numerical_cols_standard),
        ('minmax', min_max_scaler, numerical_cols_minmax),
        ('ohe', one_hot_encoder, cat_cols)
    ]
)

# Initialize SMOTE for oversampling
smote_os = SMOTE(random_state=42)

# Create the pipeline with preprocessing, SMOTE, and the classifier
pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),  # Apply preprocessing
    ('smote', smote_os),             # Apply SMOTE to balance the classes
    ('classifier', xgb.XGBClassifier(random_state=42, scale_pos_weight=10))  # Train the classifier
])

pipeline.fit(train_x, train_y)
y_pred = pipeline.predict(test_x)

print(accuracy_score(test_y, y_pred))
print(confusion_matrix(test_y, y_pred))

0.675
[[23  8]
 [ 5  4]]


In [ ]:
# Access the preprocessed data after the ColumnTransformer
train_x_preprocessed = pipeline.named_steps['preprocessor'].transform(train_x)

# Apply SMOTE to the preprocessed data
X_smote, y_smote = pipeline.named_steps['smote'].fit_resample(train_x_preprocessed, train_y)

# Check the class distribution after SMOTE
print("Class distribution after SMOTE:")
print(pd.Series(y_smote).value_counts())


Class distribution after SMOTE:
Churn
0    127
1    127
Name: count, dtype: int64
